In [32]:
import fingertips_py as ftp
import pandas as pd
import requests

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

# Area types available for each indicator

In [33]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

print(raw_available_indicator_at_area['AreaTypeId'].unique()
      ,len(raw_available_indicator_at_area['AreaTypeId'].unique())
    #   ,
    #   available_indicator_at_area['IndicatorId'].unique()
)

[402 401 102 101  15   6 221 220 204 167 166  66   7 302 219 152 301 223
 202 165 154 201  46  10 126 104 119 203 118 205 112 129 222   8   3  41] 36


In [34]:
raw_available_indicator_at_area.columns

Index(['IndicatorId', 'AreaTypeId'], dtype='object')

# Area Ids mapped to descriptive data

In [35]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

In [36]:
area_ref.columns

Index(['Id', 'Name', 'Short', 'Class', 'Sequence', 'CanBeDisplayedOnMap'], dtype='object')

# Area types available for each indicator with extra area id reference data

In [37]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True)

In [38]:
available_indicator_at_area['_merge'].unique()

['both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [39]:
available_indicator_at_area.columns

Index(['IndicatorId', 'AreaTypeId', 'Id', 'Name', 'Short', 'Class', 'Sequence',
       'CanBeDisplayedOnMap', '_merge'],
      dtype='object')

In [40]:
pd.DataFrame(available_indicator_at_area.groupby(['Name', 'Short', 'Class', 'Sequence','CanBeDisplayedOnMap'])['_merge'].count()).reset_index().sort_values(['Class', 'Sequence'], ascending=False)

,Name,Short,Class,Sequence,CanBeDisplayedOnMap,_merge
19,Lower tier local authorities (post 4/21),Districts & UAs (from Apr 2021),ua-la-composite,2021,True,381
18,Lower tier local authorities (4/20-3/21),Districts & UAs (2020/21),ua-la-composite,2020,True,173
17,Lower tier local authorities (4/19 - 3/20),Districts & UAs (2019/20),ua-la-composite,2019,True,167
20,Lower tier local authorities (pre 4/19),Districts & UAs (pre Apr 2019),ua-la-composite,2018,True,190
34,Upper tier local authorities (post 4/21),Counties & UAs (from Apr 2021),ua-county-composite,2021,True,713
33,Upper tier local authorities (4/20-3/21),Counties & UAs (2020/21),ua-county-composite,2020,True,460
32,Upper tier local authorities (4/19 - 3/20),Counties & UAs (2019/20),ua-county-composite,2019,True,646
35,Upper tier local authorities (pre 4/19),Counties & UAs (pre Apr 2019),ua-county-composite,2018,True,752
31,Sustainability and Transformation Footprints,STPs (from Apr 2020),stp-composite,2020,True,130
28,STP (120 +218),STPs (pre Apr 2020),stp-composite,2019,True,149


# Retrieve indicator data

In [41]:
# # metadata_areas = ftp.metadata.get_all_areas(is_test=False)
# # metadata_value_notes = ftp.metadata.get_all_value_notes(is_test=False)
# # metadata_profiles = ftp.metadata.get_all_profiles(is_test=False)
# # metadata_domains = ftp.metadata.get_domains_in_profile(18)
# metadata_all_indicators = ftp.metadata.get_metadata_for_all_indicators_from_csv(is_test=False)
# metadata_all_indicator_ids = metadata_all_indicators['Indicator ID'].tolist()

In [42]:
# metadata_all = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/all?include_definition=yes&include_system_content=yes')

In [43]:
# replace_comma = '%2C'
# ids_as_str = replace_comma.join([str(i) for i in metadata_all_indicator_ids])

In [44]:
# data = ftp.retrieve_data.get_all_data_for_indicators(indicators=[108,
#  113,
#  114,
#  200,
#  212],
#                                                                area_type_id=7,
#                                                                parent_area_type_id=15,
#                                                                filter_by_area_codes=None,
#                                                                is_test=False)

In [45]:
# ids_as_str

In [46]:
# data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
#                                                    area_type_id=7,
#                                                    parent_area_type_id=15,
#                                                    profile_id=None,
#                                                    include_sortable_time_periods=None,
#                                                    is_test=False)